<a href="https://colab.research.google.com/github/Yuns-u/Naver_Series_Reviews_Analysis/blob/main/Naver_Series_Reviews_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA의 목적
서비스를 평가하거나 개선점을 도출할 때 출시된 플랫폼의 리뷰나 해당 앱의 C/S의 상담 내역을 참고한다. 사용자의 경험과 느낌을 구체적으로 확인할 수 있는 리뷰는 서비스 개발 및 디자인에서 좋은 참고자료가 되기 때문에 리뷰들을 정제하면 서비스에 대한 인사이트를 파악할 때 도움이 될 것이다.

# 스크랩된 데이터 불러오기
먼저 앱스토어에서 가져온 리뷰들을 전처리하고자 한다.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv('IOS_ReviewData_naverseries.csv')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
#불필요한 칼럼인 Unnamed: 0 없애기 
df = df.drop(columns='Unnamed: 0')
df

In [ ]:
df.describe()

In [ ]:
#생각해볼 거리

#date에 시간이 필요한가?
#like, dislike에서 유의미한 정보가 있을까?
#title에 유의미한 내용을 추출할 수 있을까?
#리뷰부분에서 \n 등을 빼고 할 수 있을까?
#나의 가설은 무엇인가?
#별점이 의미가 있는가? 5점을 주는 이유: 상단으로 노출시켜서 건의하는 것을 보여주기 위한 경우가 있다.
#버전 업그레이드 될 때마다 건의사항이 바뀔 것이므로 버전 업그레이드 시기를 고려해야할 것이다.
#이 데이터는 시계열로 봐야할까?


# feature에 대해 각각 알아보기
- DATE : 리뷰 입력 시간
- STAR : 리뷰를 남긴 사용자가 해당 앱에 부여한 별점
- LIKE : 타인이 해당 리뷰가 유용하다고 평가한 횟수
- DISLIKE: 타인이 해당 리뷰가 유용하지 않다고 평가한 횟수
- TITLE : 리뷰의 제목
- REVIEW : 리뷰 내용

---
**유의사항**

- DATE: 오류가 수정된 시점 이후의 새로운 오류인지 확인할 수 있다. 시분초의 시간은 불필요할 것으로 보인다.
- STAR: 상단 노출이 되게하기 위해서 앱에 대한 평가와 무관하게 5점을 부여하는 경우가 있음. 앱에 대한 평가를 별점만으로 파악한다면 부적절한 결과가 나올 것으로 예상됨.
- LIKE: 해당 앱이 다른 사용자에게 유용하다고 생각되는 경우 다른 사용자들이 자율적으로 체크하기 때문에 범용성은 적을 것으로 보인다. 하지만 높은 좋아요 수를 가진 리뷰는 앱의 발전에 도움이 될 것으로 보인다.
- DISLIKE : 해당 앱이 다른 사용자들에게 불필요하다고 생각되는 경우 자율적으로 남길 수 있는 싫어요 표시로 마찬가지로 범용성은 적을 것으로 보인다. 하지만 지나치게 이 수치가 높은 경우 불필요한 이상치인지 판단해볼 수 있을 것이다.
- TITLE: 많은 경우 타이틀에 간략하게 문제의 원인을 적지 않고 하고 싶은 말은 본문에 작성하는 것으로 보인다. 하지만 문제의 원인을 타이틀에 간략한 키워드로 작성하는 경우가 있으므로 확인해보는 것도 좋을 것이라 생각된다.
- REVIEW: 감정과 오류에 대해 상세하게 입력할 수 있는 란으로 사용자의 앱에 대한 사용성, 느낌, 개선점 등을 파악할 수 있을 것임.


## DATE

업데이트는 20219년 11월 26일부터 24회 진행되었음.
각 업데이트 내용은 앱 기능 개선, 오류 해결, UI/UX 개선 등으로 앱의 사용성을 증진시키기 위한 내용이 많았음. 업데이트 날짜를 고려해야하지만 시분초를 고려하지 않아도 되므로 시분초를 제거하고자 함.

In [ ]:
#DATE의 시분초 없애기
df['DATE'] = pd.to_datetime(df['DATE']).dt.floor('d')
df.head()

## STAR

- H0: 별 개수에 따라 리뷰의 내용이 상이하지 않다. 부정적인 리뷰가 각 별점에 균등하게 있을 것이다.
- H1: 별 개수에 따라 리뷰의 내용이 상이하다.

리뷰에 대한 감정분석을 통해 이러한 분석도 가능할 것으로 보인다.
하지만 앱스토어에서 별 5개를 주는 경우 앱이 정말 좋아서 5개를 줄 때도 있지만 상단노출을 위해 5개를 주는 경우도 있다는 점을 유의해야 할 것이다.



In [ ]:
#별점 분포 살펴보기
import seaborn as sns

sns.countplot(df['STAR'])

위의 분포를 통해 2020년 9월20일부터 2021년 11월 6일까지의 리뷰의 별점은 위와 같은 분포를 보인다고 할 수 있다.

In [ ]:
df['STAR'].value_counts()

## LIKE
LIKE 를 받은 리뷰는 상세하거나 많은 사용자들의 공감을 얻은 리뷰일 것이다.
상위 75%까지 0이고 최고 좋아요 수가 2이므로 해당 특성으로 유의미한 개선안을 도출해내기는 어려울 것이다.

In [ ]:
df.sort_values(by='LIKE', axis=0, ascending=False)

- 좋아요를 받은 것은 좋아요 1개를 받은 3개(index=356,394,442)와 좋아요 2개를 받은 1개(index=239)이다.
- 현 시점에서 이미 수정된 오류이거나 적시성이 떨어지는 항목이다.
- 위 리뷰들은 모두 오류를 지적하는 것이므로 앱에 대한 유의미한 평가라기보다 사용자들의 니즈를 파악할 수 있는 리뷰들이 많은 핵심적인 것으로 이상치라고 볼 수 없다.
- 그러나 싫어요를 받은 평가들보다 더 정제되어있는 편이며 제목이 제목의 역할을 하는 것으로 보인다.

In [ ]:
#좋아요를 받은 리뷰들 살펴보기
print(240, df['REVIEW'].loc[240])
print(395, df['REVIEW'].loc[395])
print(443, df['REVIEW'].loc[443])
print(357, df['REVIEW'].loc[357])

In [ ]:
#싫어요보다 유용하지만 리뷰 내용보다 제목이 부실한 것은 여전하다.
#좋아요가 있는 리뷰가 4개로 전체의 1% 미만이기 때문에 차원을 줄이기 위해 없애주는 것이 좋을 것이라 판단된다.
df = df.drop(columns='LIKE')

## DISLIKE
앱을 사용하면서 사용자들이 느낀 점을 기술하는 리뷰에 대해 공감하지 못할 때 누르는 것으로 전체 500개 중에 상위 75%까지 0개, 최대 1개를 받은 것이므로 싫어요를 받은 리뷰는 스팸성 리뷰 등일 경우가 많을 것이라 예상했다.

In [ ]:
df.sort_values(by='DISLIKE', axis=0, ascending=False).head(15)

In [ ]:
#막상 확인해보니 내용과 무관하여 싫어요를 받았다고 볼 수 있는 것은 index=229의 리뷰 1개이며 나머지는 서비스에 대한 소비자의 솔직한 후기라고도 볼 수 있다.
#즉, 싫어요의 개수를 통해 리뷰의 이상치를 잡아내기는 어려운 것으로 보인다.

In [ ]:
#DISLIKE feature 없애기
df = df.drop(columns='DISLIKE')
df

## TITLE
제목 부분에는 사용자가 구체적으로 해결을 원하는 문제점의 키워드나 문제점을 한 줄로 요약하거나 의미없는 기호를 넣는 경우가 있는 것으로 보인다. 타이틀을 통해 문제점이나 해당 앱의 키워드를 살펴보고자 한다.

In [ ]:
titles = df[['DATE','TITLE']]
titles

In [ ]:
#중복되는 title이 있는지 확인
titles['TITLE'].value_counts().head(20)

In [ ]:
# 중복되지 않는 제목 살펴보기
titles['TITLE'].unique()

제목은 대체로 사용자가 리뷰를 작성할 때 가장 큰 불만점, 감정, 요구사항 등이 들어가있는 것도 있으나 리뷰에 상세히 적은 경우 제목에 불필요한 기호 등만 넣는 경우도 많아 제목을 통해 앱에 대한 사용자들의 감정이나 요구사항 등을 파악하는 것은 타당하지 않다고 생각한다.

In [ ]:
#title이 '좋아요'인 것은 6개인것들의 내용을 살펴보고자 함.
df[df['TITLE'] == '좋아요']

In [ ]:
df['REVIEW'].loc[276]

In [ ]:
df['REVIEW'].loc[385]

앱의 사용성이나 기능개선을 위해서는 오히려 리뷰의 긴 문장에서 ~되었으면 좋겠다, ~되어야 한다, ~어려워요, 단점, ~싶다 등 부정적인 어감이 들어가기 쉬운 언어에서 나올 것이라 생각한다. 제목이 리뷰의 내용의 핵심인 경우가 거의 없다는 점에서 리뷰의 제목을 통한 키워드 추출은 하지 않는 것이 적절하다 판단된다.


In [ ]:
df = df.drop(columns='TITLE')
df

# 리뷰 요약을 위해 남긴 feature
- 'DATE':후에 리뷰의 내용이 업데이트된 내용과 일치하는지 확인하기 위함
- 'REVIEW': 리뷰 내용 요약 모델에 들어갈 데이터

In [ ]:
df = df.drop(columns='STAR')
df

# 2021년 10월 26일 이후 내용 따로 빼기
iOS 기준 네이버 시리즈는 2021년 10월 25일에 마지막으로 업데이트되었다.(2021년 11월 8일 기준) 따라서 이미 업데이트되어 개선된 부분에 대해서는 높은 확률로 수정되었을 것이므로 최근 업데이트 이후의 리뷰들에서 인사이트를 따로 도출해보고자 한다.

- H0: 마지막으로 업데이트된 이후, 네이버 시리즈는 긍정적인 평가를 더 많이 받았을 것이다.
- H1: 마지막으로 업데이트된 이후, 네이버 시리즈는 긍정적인 평가를 더 많이 받았다고 할 수 없다.

In [ ]:
df.head(15)

In [ ]:
#2021년 10월 25일 이상의 리뷰들은 테스트 데이터로 따로 저장.
#2021년 10월 25일 이상의 리뷰: 총 14개
test_df = df[0:14].drop(columns='DATE')
test_df

In [ ]:
#csv로 저장하기
test_df.to_csv('iOS_test.csv')

#로컬환경으로 다운로드받기
from google.colab import files
files.download('iOS_test.csv')

In [ ]:
#나머지는 훈련 및 하이퍼파라미터 튜닝용으로 따로 저장
train_df = df[14:].drop(columns='DATE').reset_index(drop=True)
train_df

In [ ]:
#csv로 저장하기
train_df.to_csv('iOS_train.csv')

#로컬환경으로 다운로드받기
from google.colab import files
files.download('iOS_train.csv')